In [1]:
import cv2
import pandas as pd
import requests
from io import BytesIO
import numpy as np
import torch
from torchvision import models, transforms


In [2]:
df = pd.read_csv('A2_Data.csv')

In [3]:
reviews = df['Review Text']
images = df['Image']

In [4]:
import numpy as np
import cv2
from io import BytesIO
import requests
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model

# Load the ResNet50 model
base_model = VGG16(weights='imagenet', include_top=False)

# Create a new model with the VGG16 base model's output as the output
model = Model(inputs=base_model.input, outputs=base_model.output)

processed_images_list = []  # List to store processed images

for image_url in images:
    image_urls = eval(image_url)
    for url in image_urls:

        response = requests.get(url)
        image_bytes = BytesIO(response.content)
        image = cv2.imdecode(np.frombuffer(image_bytes.read(), np.uint8), cv2.IMREAD_COLOR)
        if image is None:
            processed_images_list.append(np.zeros(100352,))
            print(f"Failed to read image from {image_url}")
            continue  # Skip to the next image if loading fails

        # Basic image pre-processing
        resized_image = cv2.resize(image, (224, 224))
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Apply random flips
        if np.random.rand() > 0.5:
            resized_image = np.fliplr(resized_image)

        # Apply contrast adjustments
        # Apply brightness and contrast adjustments
        alpha = 1.0 + np.random.uniform(-0.2, 0.2)  # Brightness adjustment factor
        beta = 50 + np.random.uniform(-20, 20)      # Contrast adjustment factor
        resized_image = cv2.convertScaleAbs(resized_image, alpha=alpha, beta=beta)
        
        # Apply geometric orientation (rotation)
        angle = np.random.uniform(-30, 30)
        rows, cols, _ = resized_image.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        resized_image = cv2.warpAffine(resized_image, rotation_matrix, (cols, rows))

        resized_image = np.expand_dims(resized_image, axis=0)  # Add batch dimension
        resized_image = preprocess_input(resized_image)  # Preprocess the input for VGG16

        # Normalize the images
        resized_image /= 255.0
        # Extract features using the VGG16 model
        features = model.predict(resized_image)
        flattened_features = features.flatten()
        # Append the features to the list
        processed_images_list.append(flattened_features)

        # # Display the pre-processed image (optional)
        # print(processed_images_list)


2024-03-19 06:11:58.539765: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-19 06:11:58.539786: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-19 06:11:58.539791: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-19 06:11:58.540178: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-19 06:11:58.540699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


1/1 [==============================] - 0s 159ms/step


2024-03-19 06:11:59.183735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 13ms/step
Failed to read image from ['https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg']
Failed to read image from ['https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg']
1/1 [==============================] - 0s 13ms/step
Failed to read image from ['https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg']
Failed to read image from ['https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg']
1/1 [==============================] - 0s 12ms/step


KeyboardInterrupt: 

In [7]:
print(processed_images_list)

NameError: name 'processed_images_list' is not defined

In [5]:
from sklearn.preprocessing import StandardScaler

# Initialize a scaler
scaler = StandardScaler()

# Suppose 'all_features' is a list or array of all extracted feature vectors
dataset_features_normalized = scaler.fit_transform(processed_images_list)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (243,) + inhomogeneous part.

In [24]:
# import pickle
# with open('dataset_features_normalized.pkl', 'wb') as f:
#     pickle.dump(dataset_features_normalized, f)

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re
import math
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


 #Initialize stemmer and lemmatizer
ps = PorterStemmer()
wnl = WordNetLemmatizer()
# Function to preprocess text
def preprocess_text(text):
    # Check if text is NaN
    if isinstance(text, float) and math.isnan(text):
        return ''
    # Remove HTML tags from text
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Remove URLs from text
    text = re.sub(r'http\S+', '', text)
    # Convert text to lower case
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove punctuation
    tokens = [word for word in tokens if word.isalnum()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmed_tokens = [ps.stem(word) for word in tokens]
    # Lemmatization
    lemmatized_tokens = [wnl.lemmatize(word) for word in stemmed_tokens]
    return ' '.join(lemmatized_tokens)


[nltk_data] Downloading package punkt to /Users/jeremiah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeremiah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jeremiah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
preprocessed_texts = [preprocess_text(text) for text in reviews]

/var/folders/f5/sy3261ln1m3518y0gc0sptqr0000gn/T/ipykernel_1291/1318062935.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [28]:
import pickle
from collections import defaultdict
import math


# Calculate term frequencies (TF)
def calculate_tf(texts):
    tfs = []
    for text in texts:
        tf_dict = defaultdict(int)
        for word in text.split():
            tf_dict[word] += 1
        # tf_dict[word] = tf_dict[word] / len(text.split())
        tfs.append(tf_dict)
    return tfs

# Calculate document frequencies (DF)
def calculate_df(tfs):
    df = defaultdict(int)
    for tf in tfs:
        for word in tf.keys():
            df[word] += 1
    return df

# Calculate TF-IDF
def calculate_tfidf(tfs, dfs, N, preprocess_texts):
    tfidf = []
    for i in range(len(tfs)):
        tfidf_dict = {}
        for word, count in tfs[i].items():
            tfidf_dict[word] = count * math.log10(N / dfs[word]) / len(preprocessed_texts[i].split())
        tfidf.append(tfidf_dict)
    return tfidf

def calculate_idfs(dfs,N):
    idfs = defaultdict(int)
    for i in dfs:
        idfs[i] = math.log10(N / dfs[i])
    return idfs


tfs = calculate_tf(preprocessed_texts)
dfs = calculate_df(tfs)
N = len(preprocessed_texts)  # Number of documents
idfs = calculate_idfs(dfs,N)
tfidf_scores = calculate_tfidf(tfs, dfs, N, preprocessed_texts)
print(tfidf_scores)
with open('tfidf_scores.pkl', 'wb') as f:
    pickle.dump(tfidf_scores, f)
    

[{'love': 0.05997529699216247, 'vintag': 0.23263033265289254, 'spring': 0.2873202613314324, 'strat': 0.09043648823425417, 'good': 0.0457421853087231, 'tension': 0.13979400086720375, 'great': 0.0329900014453396, 'stabil': 0.13979400086720375, 'float': 0.1681919163520225, 'bridg': 0.09790140553051632, 'want': 0.062369467384872104, 'way': 0.0757784759919696, 'go': 0.0606729925707068}, {'work': 0.027669044352515465, 'great': 0.02151521833391713, 'guitar': 0.023835372368509123, 'bench': 0.11734652192765299, 'mat': 0.11734652192765299, 'rug': 0.08894597785046414, 'enough': 0.04792925690041559, 'abus': 0.10969038022957989, 'take': 0.10440516961273202, 'care': 0.13118593966658498, 'make': 0.038524202073615794, 'organ': 0.1000447824201731, 'workspac': 0.13043478260869565, 'much': 0.041004136898414235, 'easier': 0.07386826105808778, 'screw': 0.05021312869503232, 'wo': 0.06441243739661359, 'roll': 0.09117000056556766, 'around': 0.05250470915224983, 'color': 0.05476683958720679, 'good': 0.02983185

In [11]:
def preprocess_picture(url):
    response = requests.get(url)
    image_bytes = BytesIO(response.content)
    image = cv2.imdecode(np.frombuffer(image_bytes.read(), np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        print(f"Failed to read image from {url}")
    else:
        # Basic image pre-processing
        resized_image = cv2.resize(image, (224, 224))
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Apply random flips
        if np.random.rand() > 0.5:
            resized_image = np.fliplr(resized_image)

        # Apply contrast adjustments
        # Apply brightness and contrast adjustments
        alpha = 1.0 + np.random.uniform(-0.2, 0.2)  # Brightness adjustment factor
        beta = 50 + np.random.uniform(-20, 20)      # Contrast adjustment factor
        resized_image = cv2.convertScaleAbs(resized_image, alpha=alpha, beta=beta)
        
        # Apply geometric orientation (rotation)
        angle = np.random.uniform(-30, 30)
        rows, cols, _ = resized_image.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        resized_image = cv2.warpAffine(resized_image, rotation_matrix, (cols, rows))

        resized_image = np.expand_dims(resized_image, axis=0)  # Add batch dimension
        resized_image = preprocess_input(resized_image)  # Preprocess the input for VGG16

        # Normalize the images
        resized_image /= 255.0
        # Extract features using the VGG16 model
        features = model.predict(resized_image)
        flattened_features = features.flatten()
        return flattened_features

In [13]:
def cosine_similarity_between_two_entity(img1, img2):
    if(np.linalg.norm(img1) == 0 or np.linalg.norm(img2) == 0):
        return 0
    return np.dot(img1, img2) / (np.linalg.norm(img1) * np.linalg.norm(img2))


In [15]:
img = input()
text = input()
img = preprocess_picture(img)
if img is None:
    print(f"Failed to read image from asdasd ad {img}")
    exit()
text = preprocess_text(text)
import ast
text = text.split()
def calculate_tf_for_new_corpus(texts):
    # tfs = []
    # for text in texts:
    #     tf_dict = defaultdict(int)
    #     for word in text.split():
    #         tf_dict[word] += 1
    #     # tf_dict[word] = tf_dict[word] / len(text.split())
    #     tfs.append(tf_dict)
    # return tfs
    tf_1 = defaultdict()
    for i in range(len(texts)):
        if texts[i] in tf_1:
            tf_1[texts[i]] += 1
        else:
            tf_1[texts[i]] = 1
    
    for i in tf_1:
        tf_1[i] = tf_1[i] / len(texts)
        tf_1[i] = tf_1[i] * idfs[i]
    return tf_1
# print(text)

Newtfs = calculate_tf_for_new_corpus(text)
# print(Newtfs)
#convert newtfs into list while maintaining it's index as it is in text
# Newtfs = list(Newtfs.values())
# print(Newfs)
newtfs = [Newtfs[i] for i in text]

Similarities_for_review = []
for i in tfidf_scores:
    Similarities_for_review.append((cosine_similarity_between_two_entity(newtfs, [i[j] if j in i else 0 for j in text ])))

# get top 3 highest Similarities for review
Composite_Similarity = {}

def using_text_retrieval(top_3_review,df,Similarities_for_review, img):
    top_3_review = sorted(top_3_review)
    print(top_3_review)
    topCosinescorefortext = []
    image_urls = []
    review_text = []
    topCosinescoreforimg = []
    for i in top_3_review:
        topCosinescorefortext.append(Similarities_for_review[i])
        image_urls.append(df['Image'][i])
        a = df['Image'][i]
        review_text.append(df['Review Text'][i])
        for j in ast.literal_eval(a):
            print("Image link in top 3 text  ", j)
            print("img", img)
            list_urls = j
            image = preprocess_picture(list_urls)
            topCosinescoreforimg.append(cosine_similarity_between_two_entity(img, image))
    image_urls = sorted(image_urls,reverse = True)
    review_text = sorted(review_text,reverse = True)
    topCosinescoreforimg = sorted(topCosinescoreforimg,reverse = True)
    topCosinescorefortext = sorted(topCosinescorefortext,reverse = True)
    print(topCosinescoreforimg)
    print(topCosinescorefortext)
    print("USING TEXT RETRIEVAL")
    for i in range(3):
        print("Top ",i+1," Review")
        print("Image URL: ",image_urls[i])
        print("Review Text: ",review_text[i])
        print("Cosine Similarity Score for Image: ",topCosinescoreforimg[i])
        print("Cosine Similarity Score for Text: ",topCosinescorefortext[i])
        print("Composite Similarity Score: ",(topCosinescoreforimg[i] + topCosinescorefortext[i])/2)
        Composite_Similarity[(topCosinescoreforimg[i] + topCosinescorefortext[i])/2] = [image_urls[i],review_text[i]]
        print("\n")

        
    
# #give me top 3 highest Similarities for review
top_3_review = sorted(range(len(Similarities_for_review)), key=lambda i: Similarities_for_review[i])[-3:]

# print(top_3_review)
# print("USING TEXT RETRIEVAL")
# top_3_review_words = [reviews[i] for i in top_3_review]
# print(top_3_review)

# for i in top_3_review:
    
#     print(df['Image'][i])
#     print(df['Review Text'][i])

using_text_retrieval(top_3_review,df,Similarities_for_review, img)



1/1 [==============================] - 0s 18ms/step
[352, 528, 651]
Image link in top 3 text   https://images-na.ssl-images-amazon.com/images/I/71EgzYA-y+L._SY88.jpg
img [0.70299816 0.         0.         ... 0.         0.5828798  0.        ]
1/1 [==============================] - 0s 11ms/step
Image link in top 3 text   https://images-na.ssl-images-amazon.com/images/I/71wgWtvKmWL._SY88.jpg
img [0.70299816 0.         0.         ... 0.         0.5828798  0.        ]
1/1 [==============================] - 0s 11ms/step
Image link in top 3 text   https://images-na.ssl-images-amazon.com/images/I/61d3F7FBv+L._SY88.jpg
img [0.70299816 0.         0.         ... 0.         0.5828798  0.        ]
1/1 [==============================] - 0s 11ms/step
[0.8262099, 0.80788875, 0.7831058]
[0.8266939899196805, 0.7819437918892628, 0.7819437918892627]
USING TEXT RETRIEVAL
Top  1  Review
Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/71wgWtvKmWL._SY88.jpg']
Review Text:  Well maybe not "bette

In [16]:
Similarities = []

In [17]:
import pickle
with open ('dataset_features_normalized.pkl', 'rb') as f:
    dataset_features_normalized = pickle.load(f)


In [29]:
with open('tfidf_scores.pkl', 'rb') as f:
    tfidf_scores = pickle.load(f)

In [21]:
cumCounts={}
cumCount=0

for i in range(len(df)):
    cumCount+=df["Image"][i].count("https://")
    cumCounts[i]=cumCount
def retrieve_index(image_rank):
    for i in cumCounts:
        if image_rank < cumCounts[i]:
            return i
    return -1



In [22]:
vocab = set()
for i in tfidf_scores:
    for j in i:
        vocab.add(j)
print(len(vocab))

4438


In [30]:
def using_img_retrieval(img,top_3, Similarities,dataset_features_normalized, tfidf_scores,text):
    topCosinescorefortext = []
    image_urls = []
    review_text = []
    topCosinescoreforimg = []
    for i in top_3:
        topCosinescoreforimg.append(Similarities[i])
    Composite_similarity_score = []
    for i in range(len(top_3)):
        top_3[i] = retrieve_index(top_3[i])
    for i in top_3:
        image_urls.append(df['Image'][i])
        review_text.append(df['Review Text'][i])
    for i in range(3):
        print("Top ",i+1," Review")
        print("Image URL: ",image_urls[i])
        a = review_text[i]
        a = preprocess_text(a)
        print("Review Text: ",review_text[i])
        print("Cosine Similarity Score for Image: ",topCosinescoreforimg[i])

        a1=calculate_tf_for_new_corpus(a.split())   
        # print("text:",text)
        text1=calculate_tf_for_new_corpus((text  ))
        # print(a1)
        # print(text1)
        # print(a1, text1)
        Similarities_for_review = []
        # print(([text1[x] for x in text1], [a1[x] if x in a1 else 0 for x in text1 ] ))
        Similarities_for_review.append((cosine_similarity_between_two_entity([text1[x] for x in text1], [a1[x] if x in a1 else 0 for x in text1 ] )))
        print("Cosine Similarity Score for Text: ",Similarities_for_review)
        Composite_similarity_score.append((Similarities_for_review[-1] + topCosinescoreforimg[i])/2)
        print("Composite similarity score: ",(Similarities_for_review[-1] + topCosinescoreforimg[i])/2)
        Composite_Similarity[Similarities_for_review[-1] + topCosinescoreforimg[i]/2] = [image_urls[i],review_text[i]]


In [24]:
df.tail()

,Unnamed: 0,Image,Review Text
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the..."
999,1306,['https://images-na.ssl-images-amazon.com/imag...,This product is good and is used in profession...


In [31]:
# img = preprocess_picture(img)
print(len(dataset_features_normalized))
Similarities = []
for i in range(len(dataset_features_normalized)):
    # print(cosine_similarity_between_images(img, dataset_features_normalized[i]))
    Similarities.append(cosine_similarity_between_two_entity(img, dataset_features_normalized[i]))


CompositeSimilarity = {}


#give me top 3 highest similarities
top_3 = sorted(range(len(Similarities)), key=lambda i: Similarities[i])[-3:]
top_3.sort()

#now find the urls of the top 3
# count = 0
# for i in images:
#     image_urls = eval(i)
#     for j in image_urls:
#         for k in top_3:
#             if count == k:
#                 print(j)
#         count += 1


using_img_retrieval(img,top_3,Similarities,dataset_features_normalized, tfidf_scores,text)

1640
Top  1  Review
Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/71w8aOdrTuL._SY88.jpg']
Review Text:  I bought this bass to split time as my primary bass with my Dean Edge. This might be winning me over. The bass boost is outstanding. The active pickups really allow you to adjust to the sound you want. I recommend this for anyone. If you're a beginner  like I was not too long ago, it's an excellent bass to start with. If you're on tour and/or music is making you money, this bass will be beatiful on stage. The color is a bit darker than in the picture. But, all around, this is a great buy.
Cosine Similarity Score for Image:  0.10021603283453055
Cosine Similarity Score for Text:  [0.595367352863496]
Composite similarity score:  0.34779169284901323
Top  2  Review
Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/71YINkiMSoL._SY88.jpg']
Review Text:  Sweet upgrade to my Stratocaster. My mexican strat came with a blank chrome plate and went ahead and gave it 

In [32]:
#Arrange the top 3 composite similarity scores
Composite_Similarity = dict(sorted(Composite_Similarity.items(),reverse = True))
print("USING COMPOSITE SIMILARITY")

for i in range(3):
    print("Top ",i+1," Review")
    print("Image URL: ",Composite_Similarity[list(Composite_Similarity.keys())[i]][0])
    print("Review Text: ",Composite_Similarity[list(Composite_Similarity.keys())[i]][1])
    print("Composite Similarity Score: ",list(Composite_Similarity.keys())[i])
    print("\n")

USING COMPOSITE SIMILARITY
Top  1  Review
Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/71wgWtvKmWL._SY88.jpg']
Review Text:  Well maybe not "better", different but on the same level of awesome sound. I almost want to lie and say they suck so nobody else gets them... The sound is so round and full I couldn't believe it. I started building my own guitars and I will buy more of those soon. Put three on my home-made strat,
Composite Similarity Score:  0.8264519463415236


Top  2  Review
Image URL:  ['https://images-na.ssl-images-amazon.com/images/I/71EgzYA-y+L._SY88.jpg']
Review Text:  Simple to install and it works. I have an old Ibanez RG that I just didn't want to fuss with as far as tuning adjustments. Installed this with an extra spring and really tightened up the claw into the body, works as advertised. The coolest part is how you can set your bridge height before the final claw adjustment. Can still dive if I want but I'm just having fun playing the RG again in alt